In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Data Preprocessing (Tweepfake)

In [11]:
import os
import re

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


ABSOLUTE_PATH = "/content/drive/My Drive/deepfake_tweets"
EXPORT_DIR_PATH = os.path.join(ABSOLUTE_PATH, "data/preprocessed/")

In [12]:
# Import split dataset
train_set_path, val_set_path, test_set_path = (
    os.path.join(ABSOLUTE_PATH, "data/tweepfake_train.csv"),
    os.path.join(ABSOLUTE_PATH, "data/tweepfake_val.csv"),
    os.path.join(ABSOLUTE_PATH, "data/tweepfake_test.csv")
)

train_df, val_df, test_df = (
    pd.read_csv(train_set_path, sep=";"),
    pd.read_csv(val_set_path, sep=";"),
    pd.read_csv(test_set_path, sep=";")
)

# Concatenate split sets into a single DataFrame
df = pd.concat([train_df, val_df, test_df], ignore_index=True)

## Data Cleaning

In [13]:
def clean_data(text):
  text = re.sub(r'https?://[^\s\n\r]+', '<URL>', text) # Replace links with <URL> tag
  text = re.sub(r'#\w+', '<HASHTAG>', text)            # Replace hashtags with <HASHTAG> tag
  text = re.sub(r'@\w+', '<MENTION>', text)            # Replace mentions with <MENTION> tag
  return text

train_df['text'] = train_df['text'].apply(clean_data)
val_df['text'] = val_df['text'].apply(clean_data)
test_df['text'] = test_df['text'].apply(clean_data)

## Data Normalisation

In [14]:
def normalise_data(text):

  return text

## Exporting preprocessed dataset

In [15]:
if not os.path.exists(EXPORT_DIR_PATH):
    os.makedirs(EXPORT_DIR_PATH)

train_df.to_csv(os.path.join(EXPORT_DIR_PATH, "tweepfake_train.csv"), index=False)
val_df.to_csv(os.path.join(EXPORT_DIR_PATH, "tweepfake_val.csv"), index=False)
test_df.to_csv(os.path.join(EXPORT_DIR_PATH, "tweepfake_test.csv"), index=False)

print(f"Datasets saved to {output_dir}")


Datasets saved to /content/drive/My Drive/deepfake_tweets/data/preprocessed/
